In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.spatial import cKDTree
from shapely.geometry import Point, LineString, Polygon
from shapely.ops import nearest_points
from shapely import affinity

In [3]:
# Download CDOT streets layer
cdot_centerlines_gdf = gpd.read_file('https://data.cityofchicago.org/api/geospatial/6imu-meau?method=export&format=Original')


In [4]:
# Combine street names and types into a single column
# to match the expected format of the on/from/to street lists
gdf = cdot_centerlines_gdf.copy() 
    
gdf['On_Street'] = gdf['STREET_NAM'] + ' ' + gdf['STREET_TYP']

gdf.head(5)

,OBJECTID,FNODE_ID,TNODE_ID,TRANS_ID,PRE_DIR,STREET_NAM,STREET_TYP,SUF_DIR,STREETNAME,L_F_ADD,...,FLAG_STRIN,EWNS_DIR,EWNS_COORD,CREATE_USE,CREATE_TIM,UPDATE_USE,UPDATE_TIM,SHAPE_LEN,geometry,On_Street
0,510,10809,16581,127104,S,YALE,AVE,NaN,1782,0,...,NaN,W,232,EXISTING,1999-01-01,EXISTING,1999-01-01,220.566012,"LINESTRING (1175570.097 1863498.080, 1175577.8...",YALE AVE
1,511,6501,34082,128895,S,COTTAGE GROVE,AVE,NaN,1236,7301,...,NaN,NaN,0,EXISTING,1999-01-01,EXISTING,1999-01-01,664.774607,"LINESTRING (1182822.668 1856787.427, 1182824.9...",COTTAGE GROVE AVE
2,512,15338,22358,142645,S,CAMPBELL,AVE,NaN,1177,10801,...,NaN,W,2500,EXISTING,1999-01-01,EXISTING,1999-01-01,665.378453,"LINESTRING (1161631.239 1832936.206, 1161634.6...",CAMPBELL AVE
3,513,15799,28881,148189,S,SANGAMON,ST,NaN,1696,0,...,NaN,W,932,EXISTING,1999-01-01,EXISTING,1999-01-01,152.564966,"LINESTRING (1172013.812 1831615.472, 1171905.1...",SANGAMON ST
4,514,36407,36534,139728,W,118TH,ST,NaN,1823,1933,...,NaN,S,11800,EXISTING,1999-01-01,EXISTING,1999-01-01,332.691382,"LINESTRING (1165307.502 1826592.692, 1165260.9...",118TH ST


In [5]:
def scale_linestring(line, scale_length):
    """
    Scale a linestring to a specified length from its midpoint.
    """

    # Calculate the scaling factor
    current_length = line.length
    scaling_factor = scale_length / current_length

    # Scale the line
    midpoint = line.interpolate(0.5, normalized=True)
    scaled_line = affinity.scale(line, xfact=scaling_factor, yfact=scaling_factor, origin=midpoint)

    return scaled_line

In [6]:
def find_nearest_segments(street1_gdf, street2_gdf):
    """
    Find the nearest segment from each street based on their proximity to the other street.
    """
    # Find the segment in street1 that is closest to any point on street2
    min_distance_1 = street1_gdf.distance(street2_gdf.geometry.unary_union).min()
    nearest_segment_1 = street1_gdf[street1_gdf.distance(street2_gdf.geometry.unary_union) == min_distance_1].geometry.iloc[0]

    # Find the segment in street2 that is closest to any point on street1
    min_distance_2 = street2_gdf.distance(street1_gdf.geometry.unary_union).min()
    nearest_segment_2 = street2_gdf[street2_gdf.distance(street1_gdf.geometry.unary_union) == min_distance_2].geometry.iloc[0]
    
    return nearest_segment_1, nearest_segment_2

In [7]:
def extend_segments_to_intersection(segment1, segment2, scale_length=10560):  # 10560 feet = 2 miles
    """
    Extend the two given segments to create an intersection line.
    """
    # Create extended lines from the segments
    extended_line_1 = scale_linestring(segment1, scale_length)
    extended_line_2 = scale_linestring(segment2, scale_length)
    
    # Check intersection of the extended lines
    intersection = extended_line_1.intersection(extended_line_2)
    
    # If they intersect, return the intersection
    if not intersection.is_empty:
        return intersection
    else:
        # If they don't, return both extended lines for visualization
        return extended_line_1, extended_line_2

In [8]:
def get_intersection_point(street1_gdf, street2_gdf, scale_length=2640):  # 2640 ft = 1/2 mile
    """
    Return the intersection point of two streets. If they don't intersect, find the closest features
    and create a virtual intersection by extending the features to the specified scale_length.
    """
    intersection = street1_gdf.geometry.unary_union.intersection(street2_gdf.geometry.unary_union)
    
    # If intersection exists and is a point, return it
    if not intersection.is_empty:
        if intersection.geom_type == "Point":
            return intersection
        elif intersection.geom_type == "MultiPoint":
            return intersection[0]
    
    # If no intersection, find the closest points and create a virtual intersection
    nearest_segment_1, nearest_segment_2 = find_nearest_segments(street1_gdf, street2_gdf)
    
    virtual_intersection = extend_segments_to_intersection(nearest_segment_1, nearest_segment_2)
    
    return virtual_intersection


In [9]:
def filter_segments_between_points(on_street_gdf, from_intersection, to_intersection):
    """
    Filter the on_street segments based on the orientation of the line formed by the intersections.
    """
    # Determine the orientation of the intersection line
    delta_x = abs(to_intersection.x - from_intersection.x)
    delta_y = abs(to_intersection.y - from_intersection.y)
    
    filtered_segments = []
    
    # If intersection_line is oriented more in the x direction
    if delta_x > delta_y:
        min_x, max_x = sorted([from_intersection.x, to_intersection.x])
        for index, row in on_street_gdf.iterrows():
            midpoint_x = row['geometry'].centroid.x
            if min_x <= midpoint_x <= max_x:
                filtered_segments.append(row['geometry'])
    # If intersection_line is oriented more in the y direction
    else:
        min_y, max_y = sorted([from_intersection.y, to_intersection.y])
        for index, row in on_street_gdf.iterrows():
            midpoint_y = row['geometry'].centroid.y
            if min_y <= midpoint_y <= max_y:
                filtered_segments.append(row['geometry'])
                
    # Convert the list of filtered segments to a GeoDataFrame
    filtered_gdf = gpd.GeoDataFrame(geometry=filtered_segments, crs=on_street_gdf.crs)
    
    return filtered_gdf

In [23]:
    
def extract_street_segments(gdf, on_street, from_street, to_street):
    '''
    Extract the segment of on_street that is between its intersection with from_street and to_street.
    
    on_street, from_street, and to_street are strings representing cleaned official street names found
    in the gdf.  (For example, "Madison St")
    
    gdf is the CDOT Street Centerline shapefile downloaded and converted to a 
    GeoDataFrame.
    '''

    # This version is for the CDOT map base layer, downloaded as a shapefile.
    # It has separate street_nam and street_typ fields.

    # Filter the GeoDataFrame for the given streets, ignoring case 
    on_street_gdf = gdf[gdf['On_Street'].str.lower() == on_street.lower()]
    
    from_street_gdf = gdf[gdf['On_Street'].str.lower() == from_street.lower()]
    
    to_street_gdf = gdf[gdf['On_Street'].str.lower() == to_street.lower()]
    
    # Get the intersection points
    on_from_point = get_intersection_point(on_street_gdf, from_street_gdf)
    on_to_point = get_intersection_point(on_street_gdf, to_street_gdf)
    
    # Filter the segments based on the orientation of the line formed by the intersections
    filtered_segments_gdf = filter_segments_between_points(on_street_gdf, on_from_point, on_to_point)
    
    # dissolve so the entire corridor is one feature
    dissolved_gdf = filtered_segments_gdf.dissolve()
    
    # add on/from/to street data to the feature
    dissolved_gdf['On'] = on_street
    dissolved_gdf['From'] = from_street
    dissolved_gdf['To'] = to_street

    return dissolved_gdf



In [25]:
## Run code to test ##

on_street = ('Madison St')
from_street = ('Clark St')
to_street = ('Lawndale Ave')

test_gdf = extract_street_segments(gdf, on_street, from_street, to_street)

test_gdf.explore()

/opt/homebrew/Caskroom/miniforge/base/envs/cta-bus-tracker-env/lib/python3.11/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
